In [18]:
#輸出csv字串,並且儲存為檔案
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import os
from datetime import datetime
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
# response = requests.get('https://www.merit-times.com.tw/NewsPage.aspx?unid=903451')
response = requests.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
soup = BeautifulSoup(response.text,"html.parser")
body_content = soup.body
for script_or_style in body_content(['script','style']):
    script_or_style.extract()

table_content = body_content.find(title='牌告匯率')
table_lines = table_content.get_text(separator='\n')
cleaned_content = "\n".join(
    [line.strip() for line in table_lines.splitlines() if line.strip()]
    )


model = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction='''
    1. 你的任務是取出指定的內容,並輸出成csv的格式
    2. 只要輸出csv格式的字串,不要有多餘的文字

    請依照下面的指示:    
    ### 如果找到下面的樣本:
    ```
    澳幣 (AUD)
    澳幣 (AUD)
    20.15
    20.93
    20.365
    20.71
    ```

    ### 輸出的csv格式:
    1. 第1欄:有欄位名稱
    2. 欄位名稱
    - 幣別:str國幣的名稱
    - 幣別代碼:str國家的代碼
    - `現金匯率(本行買入)`:float,無法轉換請用null
    - `現金匯率(本行賣出)`:float,無法轉換請用null
    - `即期匯率(本行買入)`:float,無法轉換請用null
    - `即期匯率(本行賣出)`:float,無法轉換請用null

    ### 輸出的樣本
    ```
    幣別,幣別代碼,現金匯率(本行買入),現金匯率(本行賣出),即期匯率(本行買入),即期匯率(本行賣出)
    澳幣,AUD,20.15,20.93,20.365,20.71
    ```  
    '''
)

response = model.generate_content(cleaned_content)
result_text = response.text

def text_to_csv(csv_text:str):
    '''
    將傳入的csv格式字串,儲存為以現在日期為檔案名稱的csv檔
    '''
    current = datetime.now()
    filename = current.strftime("")
    filename = current.strftime("%Y_%m_%d") + ".csv"
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(csv_text)

text_to_csv(result_text)





In [10]:
model = genai.GenerativeModel(
    "gemini-1.5-pro",
    system_instruction='''
    請將輸入的csv格式的字串,儲存為`現在日期.csv`
    ''',
    tools='code_execution'
)

response = model.generate_content(result_text)


In [16]:
generated_text = response.candidates[0].content

AttributeError: Unknown field for Content: split